In [1]:
# 總時間約25分鐘
# 上市櫃公司平均薪資要高於多少? 請在這邊調整，單位是千元
salary_criteria = 1200

In [2]:
import re
import requests
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time
import difflib
import winsound

In [3]:
start = time.time()

In [4]:
# 讀取上市櫃薪資排行榜，找出高薪資公司
# 不論員工人數，所以控股公司也會被列入
salary = pd.read_excel('salary_data.xlsx', sheet_name='上市櫃5年薪資中位數')
salary_high = salary[salary['非擔任主管職務之全時員工資訊-員工薪資-中位數(仟元/人)-111年'] > salary_criteria]
high_company_list = salary_high['公司全名'].to_list()

c:\Users\Cooper\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
c:\Users\Cooper\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [5]:
# remove nan
# high_company_list_original變數不可覆蓋，要給後面比對公司用
high_company_list_original = [x for x in high_company_list if str(x) != 'nan']

# 只取前面5個字，模糊搜尋，比較容易找到公司名稱
# 日月光投資控股股份有限公司 - 全名會找不到，所以只取前5個字，識別機率更高
high_company_list = [i[:5] for i in high_company_list_original]
search_company = ['https://www.104.com.tw/company/search/?keyword={}'.format(i) for i in high_company_list]

In [6]:
# 使用difflib，找出最相似的公司名稱
def similar_diff_qk_ratio(str1, str2):
    return difflib.SequenceMatcher(None, str1, str2).quick_ratio()

In [7]:
# 自動翻頁
def scroll(driver, timeout):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script('return document.body.scrollHeight')

    while True:
        # Scroll down to bottom
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')

        # Wait to load page
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

In [8]:
# 抓公司專頁的網址
company_dict = {}

for original_company_name, url in zip(high_company_list_original, search_company):
    res = requests.get(url)
    soup = bs(res.text, 'lxml')

    # 搜尋結果，只抓前面6個公司名稱進行比對，抓太多可能會識別到其他更相似公司
    # 日月光投資控股股份有限公司，每次搜尋排名都不一樣，容易抓錯
    catch_company = []
    for _ in soup.select('div > a.d-none.d-md-inline.jb-link.jb-link-blue.jb-link-blue--visited.h2.mb-1')[:6]:
        catch_company.append(_.text)

    similar_company = []
    for random_name in catch_company:
        similar_company.append(similar_diff_qk_ratio(random_name, original_company_name))

    # 抓取正確的公司名稱及網址
    try:
        # 返回最大值的索引，即最相似的公司名稱順位
        index = similar_company.index(max(similar_company))

        correct_name = soup.select('div > a.d-none.d-md-inline.jb-link.jb-link-blue.jb-link-blue--visited.h2.mb-1')[index].text
        correct_page = soup.select('div > a.d-none.d-md-inline.jb-link.jb-link-blue.jb-link-blue--visited.h2.mb-1')[index]['href']
        company_dict.update({correct_name : correct_page})
        print('爬完' + '\t\t' + correct_name)
    except:
        pass
        print('找不到' + '\t' + original_company_name)
        
print('共' + str(len(company_dict)) + '家公司')

爬完		信驊科技股份有限公司
爬完		長榮海運股份有限公司
爬完		聯發科技股份有限公司
爬完		瑞鼎科技股份有限公司
爬完		矽創電子股份有限公司
爬完		聯詠科技股份有限公司
爬完		瑞昱半導體股份有限公司
爬完		杰力科技股份有限公司
爬完		力旺電子股份有限公司
爬完		陽明海運股份有限公司
爬完		晶豪科技股份有限公司
爬完		台灣積體電路製造股份有限公司(台積電)
爬完		九齊科技股份有限公司
爬完		原相科技股份有限公司
爬完		致新科技股份有限公司
爬完		萬海航運股份有限公司
爬完		天鈺科技股份有限公司
爬完		創意電子股份有限公司
爬完		昇佳電子股份有限公司
爬完		凌通科技股份有限公司
爬完		智原科技股份有限公司
爬完		晶心科技股份有限公司
爬完		智擎生技製藥股份有限公司
爬完		鈊象電子股份有限公司
爬完		晶相光電股份有限公司
爬完		M31 Technology Corporation_円星科技股份有限公司
爬完		凌陽創新科技股份有限公司
爬完		阜聯電子股份有限公司
爬完		聯陽半導體股份有限公司
爬完		祥碩科技股份有限公司
爬完		大中積體電路股份有限公司
爬完		亞信電子股份有限公司
爬完		是方電訊股份有限公司(中華電信關係企業)
爬完		鴻海精密工業股份有限公司
爬完		創惟科技股份有限公司
爬完		南帝化學工業股份有限公司
爬完		威鋒電子股份有限公司
爬完		和泰汽車股份有限公司
爬完		力智電子股份有限公司
爬完		群光電子股份有限公司
爬完		愛普科技股份有限公司
爬完		弘塑科技股份有限公司
爬完		阜聯電子股份有限公司
爬完		揚智科技股份有限公司
爬完		盛群半導體股份有限公司
爬完		鴻準精密工業股份有限公司
爬完		九暘電子股份有限公司
爬完		亞洲電材股份有限公司
爬完		松翰科技股份有限公司
爬完		華邦電子股份有限公司
爬完		亞泰影像科技股份有限公司
爬完		義隆電子股份有限公司
爬完		華固建設股份有限公司
爬完		來頡科技股份有限公司
爬完		精成科技股份有限公司
爬完		致伸科技股份有限公司
爬完		精拓科技股份有限公司
爬完		群光電能科技股份有限公司
爬完		敦泰電子股份有限公司
爬完		智易科技股份有限公司
爬完		中華電信股份有限公司
爬完		四維航業股份有限公司
爬完		蔚

In [9]:
chrome_options = Options()
chrome_options.add_argument('--headless')           # 指定headless，并且禁用gpu
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('window-size=1920,1080') # 設定瀏覽器尺寸，假如無法滾動的話

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get("https://www.104.com.tw/jobs/main/")
# driver.maximize_window()

In [10]:
company = []
job = []
location = []
salary = []
requirement = []
education = []
descirption = []
link = []

In [11]:
# 頁碼規則
# https://www.104.com.tw/company/12noppgo?job=&roleJobCat=0_0&area=0&page=1&pageSize=20&order=8&asc=0&jobsource=cs_custlist&tab=job
# https://www.104.com.tw/company/12noppgo?job=&roleJobCat=0_0&area=0&page=2&pageSize=20&order=8&asc=0&jobsource=cs_custlist&tab=job

In [12]:
for name, url in zip(company_dict.keys(), company_dict.values()):
    driver.get(url)
    time.sleep(1)   # 等待網頁載入

    # 預設顯示20個工作，找出總頁數
    try:
        text = driver.find_element(By.CSS_SELECTOR, 'div.col.main > div.joblist.rounded > div.joblist__footer > div > div:nth-child(2) > label > select').text
        total_page = int(re.findall(r'\d+', text)[-1])
    except:
        #如果找不到每筆100頁選單，則總頁數為1頁
        total_page = 1

    print(name + '\t' + '總頁數' + str(total_page))

    # 先寫爬取第一頁，之後再翻頁
    for i in range(1, total_page+1):
        driver.get(url + '?job=&roleJobCat=0_0&area=0&page={}&pageSize=20&order=8&asc=0&jobsource=cs_custlist&tab=job'.format(i))
        # 自動翻頁
        scroll(driver, 1)
        for i in range(1, 105): # 避免中間有廣告或是在頂端的急徵工作，所以最大值>100
            try:
                job_tag = driver.find_element(By.CSS_SELECTOR, "div:nth-child({}) > div > div.info.col > div > div.info-job.text-break.mb-2 > a".format(i)).text
                link_tag = driver.find_element(By.CSS_SELECTOR, "div:nth-child({}) > div > div.info.col > div > div.info-job.text-break.mb-2 > a".format(i)).get_attribute("href")
                location_tag = driver.find_element(By.CSS_SELECTOR, 'div:nth-child({}) > div > div.info.col > div > div.info-tags.gray-deep-dark > span:nth-child(1)'.format(i)).text
                requirement_tag = driver.find_element(By.CSS_SELECTOR, 'div:nth-child({}) > div > div.info.col > div > div.info-tags.gray-deep-dark > span:nth-child(2)'.format(i)).text
                education_tag = driver.find_element(By.CSS_SELECTOR, 'div:nth-child({}) > div > div.info.col > div > div.info-tags.gray-deep-dark > span:nth-child(3)'.format(i)).text
                descirption_tag = driver.find_element(By.CSS_SELECTOR, 'div:nth-child({}) > div > div.info.col > div > div.info-description.text-gray-darker.t4.text-break.mt-2.position-relative.info-description__line2'.format(i)).text
                salary_tag = driver.find_element(By.CSS_SELECTOR, 'div:nth-child({}) > div > div.info.col > div > div.info-othertags > span'.format(i)).text

                job.append(job_tag)
                link.append(link_tag)
                location.append(location_tag)
                requirement.append(requirement_tag)
                education.append(education_tag)
                descirption.append(descirption_tag)
                salary.append(salary_tag)
                company.append(name)
            except:
                pass

    print(name + '\t' + '爬完')

信驊科技股份有限公司	總頁數1
信驊科技股份有限公司	爬完
長榮海運股份有限公司	總頁數1
長榮海運股份有限公司	爬完
聯發科技股份有限公司	總頁數7
聯發科技股份有限公司	爬完
瑞鼎科技股份有限公司	總頁數5
瑞鼎科技股份有限公司	爬完
矽創電子股份有限公司	總頁數2
矽創電子股份有限公司	爬完
聯詠科技股份有限公司	總頁數11
聯詠科技股份有限公司	爬完
瑞昱半導體股份有限公司	總頁數21
瑞昱半導體股份有限公司	爬完
杰力科技股份有限公司	總頁數1
杰力科技股份有限公司	爬完
力旺電子股份有限公司	總頁數1
力旺電子股份有限公司	爬完
陽明海運股份有限公司	總頁數1
陽明海運股份有限公司	爬完
晶豪科技股份有限公司	總頁數3
晶豪科技股份有限公司	爬完
台灣積體電路製造股份有限公司(台積電)	總頁數11
台灣積體電路製造股份有限公司(台積電)	爬完
九齊科技股份有限公司	總頁數1
九齊科技股份有限公司	爬完
原相科技股份有限公司	總頁數2
原相科技股份有限公司	爬完
致新科技股份有限公司	總頁數1
致新科技股份有限公司	爬完
萬海航運股份有限公司	總頁數2
萬海航運股份有限公司	爬完
天鈺科技股份有限公司	總頁數2
天鈺科技股份有限公司	爬完
創意電子股份有限公司	總頁數1
創意電子股份有限公司	爬完
昇佳電子股份有限公司	總頁數1
昇佳電子股份有限公司	爬完
凌通科技股份有限公司	總頁數1
凌通科技股份有限公司	爬完
智原科技股份有限公司	總頁數3
智原科技股份有限公司	爬完
晶心科技股份有限公司	總頁數2
晶心科技股份有限公司	爬完
智擎生技製藥股份有限公司	總頁數1
智擎生技製藥股份有限公司	爬完
鈊象電子股份有限公司	總頁數4
鈊象電子股份有限公司	爬完
晶相光電股份有限公司	總頁數1
晶相光電股份有限公司	爬完
M31 Technology Corporation_円星科技股份有限公司	總頁數1
M31 Technology Corporation_円星科技股份有限公司	爬完
凌陽創新科技股份有限公司	總頁數1
凌陽創新科技股份有限公司	爬完
阜聯電子股份有限公司	總頁數1
阜聯電子股份有限公司	爬完
聯陽半導體股份有限公司	總頁數1
聯陽半導體股份有限公司	爬完
祥碩科技股份有限公司	總頁數1
祥碩科技股份有限公司	爬完
大中積體電路股份有限公司	

In [13]:
data = {
    'company':      company,
    'job_name':     job,
    'location':     location,
    'salary':       salary,
    'requirement':  requirement,
    'education':    education,
    'descirption':  descirption,
    'link':         link,
}

df = pd.DataFrame(data)
df.drop_duplicates(subset='link',inplace=True)   #link一樣，內容就會一樣，刪除重複的置頂急徵工作
df.to_excel('104_上市櫃高薪工作.xlsx', sheet_name= '上市櫃高薪工作')
df

,company,job_name,location,salary,requirement,education,descirption,link
0,信驊科技股份有限公司,Embedded Linux System Engineer,新竹市,待遇面議,3年以上,碩士,"1. Linux kernel, device driver development on ...",https://www.104.com.tw/job/7dny7?jobsource=cs_...
1,信驊科技股份有限公司,Security System Engineer,新竹市,待遇面議,3年以上,碩士,1. Familiar with Platform Firmware Resilience ...,https://www.104.com.tw/job/7do09?jobsource=cs_...
2,信驊科技股份有限公司,BMC 軟體系統應用工程師,新竹市,待遇面議,3年以上,碩士,"1. 熟悉 BMC Interface such as LPC/eSPI ,Ethernet...",https://www.104.com.tw/job/7dzvl?jobsource=cs_...
3,信驊科技股份有限公司,firmware 軟韌體開發工程師,新竹市,待遇面議,5年以上,碩士,The ASPEED firmware development team is lookin...,https://www.104.com.tw/job/637mv?jobsource=cs_...
4,信驊科技股份有限公司,Embedded Linux System Engineer (MCU),新竹市,待遇面議,3年以上,碩士,"1. Linux kernel, device driver development on ...",https://www.104.com.tw/job/7hiun?jobsource=cs_...
...,...,...,...,...,...,...,...,...
5761,立積電子股份有限公司,Technical Marketing (長期美國出差),美加地區,待遇面議,6年以上,大學以上,I. Oversee day-to-day II. Managing project iss...,https://www.104.com.tw/job/7anpp?jobsource=cs_...
5762,立積電子股份有限公司,Sales Representative (North American),台北市內湖區,待遇面議,經歷不拘,碩士以上,*Develop and Maintain Customer relationships *...,https://www.104.com.tw/job/7ebpe?jobsource=cs_...
5763,立積電子股份有限公司,測試助理工讀(09:00-18:00),新竹縣竹北市,"月薪27,400~27,500元",經歷不拘,高中,1. 焊接PCBA 測試板或者解焊零件 2. 測試IC 功能性能測試FT test 3. A...,https://www.104.com.tw/job/7pwjh?jobsource=cs_...
5764,立積電子股份有限公司,資深生產企劃副/經理,新竹縣竹北市,待遇面議,8年以上,大學,1. 評估客戶需求之產能規劃並分配至各外包封測廠 2. 具合約審查、成本分析及議價的能力。 ...,https://www.104.com.tw/job/74ayr?jobsource=cs_...


In [14]:
print(len(company))
print(len(job))
print(len(location))
print(len(salary))
print(len(requirement))
print(len(education))
print(len(descirption))
print(len(link))

5766
5766
5766
5766
5766
5766
5766
5766


In [15]:
end = time.time()
print('完成共花費',round((end - start)/60,2),'分')

完成共花費 24.19 分


In [16]:
winsound.Beep(300,500)
winsound.Beep(360,500)
winsound.Beep(500,500)
winsound.PlaySound('alert', winsound.SND_ASYNC)